In [1]:
# -*- coding: utf-8 -*-
# 作者: xcl
# 时间: 2019/8/1 21:51
import pandas as pd
import keras
from keras.layers import Input, Embedding, LSTM, Dense, concatenate, core, add
from keras.models import Model
import os
import numpy as np
from sklearn.preprocessing import MinMaxScaler

Using TensorFlow backend.


In [2]:
# 读取
input_path = 'D:\\全2018_日出足够.xlsx'
data_all = pd.read_excel(input_path, index_col='日期')
data_all = data_all.dropna()


In [3]:
data_ts_df = data_all[['tm_mon', 'tm_mday', 'tm_wday', 'tm_yday', 'tm_week','id']]
# 虚拟变量
for ccc in data_ts_df.columns:
    data_ts_df[ccc] = data_ts_df[ccc].map(lambda x : str(x))
data_get_dummies1 = pd.get_dummies(data_ts_df[['tm_mon']],drop_first=True)
data_get_dummies2 = pd.get_dummies(data_ts_df[['tm_mday']], drop_first=True)
data_get_dummies3 = pd.get_dummies(data_ts_df[['id']], drop_first=True)
data_dummies = pd.concat([data_get_dummies1,data_get_dummies2,data_get_dummies3,data_ts_df[['tm_mon']]], axis =1)


D:\DataScience\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [31]:
list1 = []
for ccc in data_dummies.columns:
    
    if 'id' in ccc:
        list1.append(ccc)
print(list1)
print(len(list1))

['id_1', 'id_100', 'id_101', 'id_102', 'id_103', 'id_104', 'id_105', 'id_106', 'id_107', 'id_108', 'id_109', 'id_110', 'id_111', 'id_112', 'id_113', 'id_114', 'id_115', 'id_116', 'id_117', 'id_118', 'id_119', 'id_120', 'id_121', 'id_122', 'id_123', 'id_124', 'id_125', 'id_126', 'id_127', 'id_128', 'id_129', 'id_130', 'id_131', 'id_132', 'id_133', 'id_134', 'id_135', 'id_136', 'id_137', 'id_138', 'id_139', 'id_140', 'id_141', 'id_142', 'id_143', 'id_144', 'id_145', 'id_146', 'id_147', 'id_148', 'id_149', 'id_150', 'id_151', 'id_2', 'id_26', 'id_28', 'id_3', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'id_39', 'id_40', 'id_41', 'id_42', 'id_43', 'id_44', 'id_45', 'id_46', 'id_47', 'id_48', 'id_49', 'id_50', 'id_51', 'id_52', 'id_53', 'id_54', 'id_55', 'id_56', 'id_57', 'id_58', 'id_59', 'id_60', 'id_61', 'id_62', 'id_63', 'id_64', 'id_65', 'id_66', 'id_67', 'id_68', 'id_69', 'id_70', 'id_71', 'id_72', 'id_73', 'id_74', 'id_75', 'id_76', 'id_77', 'id_78', 'id_79', 'id_8

In [7]:
# 去掉无用列
data_to_std = data_all.drop(['tm_mon', 'tm_mday', 'tm_wday', 'tm_yday', 'tm_week','id'], axis=1)

In [8]:
# 标准化
from sklearn.preprocessing import MinMaxScaler
data_to_std2= MinMaxScaler().fit_transform(data_to_std)
data_to_std2 = pd.DataFrame(data_to_std2)
data_to_std2 = data_to_std2.set_index(data_to_std.index)
data_to_std2.columns = data_to_std.columns
print(data_to_std2.shape)
print(data_ts_df.shape)
data_out = pd.concat([data_dummies, data_to_std2],join='outer', axis=1)

(29753, 105)
(29753, 6)


In [20]:
# 划分
data_test = data_out[(data_out['tm_mon']=="2")|
                     (data_out['tm_mon']=="5")|
                    (data_out['tm_mon']=="8")|
                    (data_out['tm_mon']=="11")]
data_train = data_out[(data_out['tm_mon']=="1")|
                     (data_out['tm_mon']=='3')|
                    (data_out['tm_mon']=='5')|
                    (data_out['tm_mon']=='6')|
                     (data_out['tm_mon']=='7')|
                    (data_out['tm_mon']=='9')|
                    (data_out['tm_mon']=='10')|
                     (data_out['tm_mon']=='12')]

In [21]:
# AOD
data_aod_test = data_test[['AOD_0']]
data_aods_test = data_test[[
                     'AOD_1',
                     'AOD_2',
                     'AOD_3',
                     'AOD_4',
                     'AOD_5',
                     'AOD_6',
                     'AOD_7',
                     'AOD_8',
                     'AOD_9',
                     'AOD_10',
                     'AOD_11',
                     'AOD_12',
                     'AOD_13',
                     'AOD_14',
                     'AOD_15',
                     'AOD_16']]

data_aod_train = data_train[['AOD_0']]
data_aods_train = data_train[[
                     'AOD_1',
                     'AOD_2',
                     'AOD_3',
                     'AOD_4',
                     'AOD_5',
                     'AOD_6',
                     'AOD_7',
                     'AOD_8',
                     'AOD_9',
                     'AOD_10',
                     'AOD_11',
                     'AOD_12',
                     'AOD_13',
                     'AOD_14',
                     'AOD_15',
                     'AOD_16']]

In [22]:
# 气象
data_sky_test = data_test[['apparentTemperatureHigh',
                     'apparentTemperatureLow',
                     'apparentTemperatureMax',
                     'apparentTemperatureMin',
                     'cloudCover',
                     'dewPoint',
                     'humidity',
                     'pressure',
                     'sunTime',
                     'temperatureHigh',
                     'temperatureLow',
                     'temperatureMax',
                     'temperatureMin',
                     'uvIndex',
                     'visibility',
                     'windBearing',
                     'windGust',
                     'windSpeed',
                     'apparentTemperature',
                     'temperature']]

data_sky_train = data_train[['apparentTemperatureHigh',
                     'apparentTemperatureLow',
                     'apparentTemperatureMax',
                     'apparentTemperatureMin',
                     'cloudCover',
                     'dewPoint',
                     'humidity',
                     'pressure',
                     'sunTime',
                     'temperatureHigh',
                     'temperatureLow',
                     'temperatureMax',
                     'temperatureMin',
                     'uvIndex',
                     'visibility',
                     'windBearing',
                     'windGust',
                     'windSpeed',
                     'apparentTemperature',
                     'temperature']]
print(data_sky_test.index)

Index(['2018-08-15', '2018-08-16', '2018-08-17', '2018-08-18', '2018-08-19',
       '2018-08-20', '2018-08-21', '2018-08-22', '2018-08-23', '2018-08-24',
       ...
       '2018-05-06', '2018-05-07', '2018-05-08', '2018-05-09', '2018-05-10',
       '2018-05-11', '2018-05-12', '2018-05-13', '2018-05-14', '2018-05-15'],
      dtype='object', name='日期', length=9857)


In [23]:
# 时空特征
data_ts_test = data_test[list1]
print(data_ts_test.index)
data_ts_train = data_train[list1]

Index(['2018-08-15', '2018-08-16', '2018-08-17', '2018-08-18', '2018-08-19',
       '2018-08-20', '2018-08-21', '2018-08-22', '2018-08-23', '2018-08-24',
       ...
       '2018-05-06', '2018-05-07', '2018-05-08', '2018-05-09', '2018-05-10',
       '2018-05-11', '2018-05-12', '2018-05-13', '2018-05-14', '2018-05-15'],
      dtype='object', name='日期', length=9857)


In [24]:
# 时滞
data_t1_test = data_test[['AOD_0_T1',
                    'apparentTemperatureHigh_T1',
                    'apparentTemperatureLow_T1',
                    'apparentTemperatureMax_T1',
                    'apparentTemperatureMin_T1',
                    'cloudCover_T1',
                    'dewPoint_T1',
                    'humidity_T1',
                    'pressure_T1',
                    'sunriseTime_T1',
                    'sunsetTime_T1',
                    'temperatureHigh_T1',
                    'temperatureLow_T1',
                    'temperatureMax_T1',
                    'temperatureMin_T1',
                    'uvIndex_T1',
                    'visibility_T1',
                    'windBearing_T1',
                    'windGust_T1',
                    'windSpeed_T1',
                    'apparentTemperature_T1',
                    'temperature_T1']]

data_t1_train = data_train[['AOD_0_T1',
                    'apparentTemperatureHigh_T1',
                    'apparentTemperatureLow_T1',
                    'apparentTemperatureMax_T1',
                    'apparentTemperatureMin_T1',
                    'cloudCover_T1',
                    'dewPoint_T1',
                    'humidity_T1',
                    'pressure_T1',
                    'sunriseTime_T1',
                    'sunsetTime_T1',
                    'temperatureHigh_T1',
                    'temperatureLow_T1',
                    'temperatureMax_T1',
                    'temperatureMin_T1',
                    'uvIndex_T1',
                    'visibility_T1',
                    'windBearing_T1',
                    'windGust_T1',
                    'windSpeed_T1',
                    'apparentTemperature_T1',
                    'temperature_T1']]

In [25]:
# NDVI
data_ndvi_test = data_test[['NDVI_0','AOD_1',
                     'AOD_2',
                     'AOD_3',
                     'AOD_4',
                     'AOD_5',
                     'AOD_6',
                     'AOD_7',
                     'AOD_8',
                     'AOD_9',
                     'AOD_10',
                     'AOD_11',
                     'AOD_12',
                     'AOD_13',
                     'AOD_14',
                     'AOD_15',
                     'AOD_16']]
data_ndvi_train = data_train[['NDVI_0','AOD_1',
                     'AOD_2',
                     'AOD_3',
                     'AOD_4',
                     'AOD_5',
                     'AOD_6',
                     'AOD_7',
                     'AOD_8',
                     'AOD_9',
                     'AOD_10',
                     'AOD_11',
                     'AOD_12',
                     'AOD_13',
                     'AOD_14',
                     'AOD_15',
                     'AOD_16']]

In [26]:
# 污染物
data_pm_test = data_test[['PM25']]
data_pm_train = data_train[['PM25']]

In [27]:
# 第一个模块 当日天气模块
#
# 输入层
Meteorology_input = Input(shape=(len(data_sky_test.columns),), name="Meteorology_input")
AODs_input = Input(shape=(len(data_aod_test.columns),), name="AODs_input")
# 因素融合层
meteorology_concat = concatenate([Meteorology_input, AODs_input])
# 全连接层
meteorology_x = Dense(
    24,
    activation=keras.layers.LeakyReLU(alpha=0.4),
    name="FullConnectionMA_1")(meteorology_concat)
# 残差层
meteorology_residual_connection = Dense(
    16,
    activation=keras.layers.LeakyReLU(alpha=0.3),
    name="ResidualConnectionMA")(meteorology_x)
meteorology_residual_connection = Dense(
    24,
    activation=keras.layers.LeakyReLU(alpha=0.4),
    name="FullConnectionMA_RC")(meteorology_residual_connection)
meteorology_residual_connection = add(
    [meteorology_x, meteorology_residual_connection], name="ResidualConnectionMA_Add")
# 全连接层
meteorology_x = Dense(
    12,
    activation=keras.layers.LeakyReLU(alpha=0.4),
    name="FullConnectionMA_2")(meteorology_residual_connection)
# Dropout
meteorology_y = core.Dropout(
    rate=0.5, name="Meteorology_Module")(meteorology_x)
# 模型层
model_meteorology = Model(
    inputs=[
        Meteorology_input,
        AODs_input],
    outputs=meteorology_y)
# 第二个模块 时滞天气模块

Weather_input = Input(shape=(len(data_t1_test.columns),), name="Weather_input")  # 只添加新的因素即可
# AODs_input = Input(shape=(17,), name="AODs_input")
# 因素融合层
weather_concat = concatenate([Weather_input, AODs_input])
# 全连接层
weather_x = Dense(
    24,
    activation=keras.layers.LeakyReLU(alpha=0.4),
    name="FullConnectionWA_1")(weather_concat)
# 残差层
weather_residual_connection = Dense(
    12,
    activation=keras.layers.LeakyReLU(alpha=0.4),
    name="ResidualConnectionWA")(weather_x)
weather_residual_connection = Dense(
    24,
    activation=keras.layers.LeakyReLU(alpha=0.4),
    name="FullConnectionWAForRC")(weather_residual_connection)
weather_residual_connection = add(
    [weather_x, weather_residual_connection], name="ResidualConnectionWA_Add")
# 全连接层
weather_x = Dense(
    12,
    activation=keras.layers.LeakyReLU(alpha=0.4),
    name="FullConnectionWA_2")(weather_residual_connection)
# Dropout
# use `rate` instead of `keep_prob`. r=1-p.
weather_y = core.Dropout(rate=0.5, name="Weather_Module")(weather_x)

# 模型层
model_weather = Model(inputs=[Weather_input, AODs_input], outputs=weather_y)


# 第三个模块 其他空气污染物模块 更改为ndvi
# 输入层
OtherPollution_input = Input(shape=(len(data_ndvi_test.columns),))
# 因素融合层
otherpollution_concat = concatenate([OtherPollution_input, AODs_input])
# 全连接层
otherpollution_x = Dense(
    24,
    activation=keras.layers.LeakyReLU(alpha=0.4),
    name="FullConnectionOPA_1")(otherpollution_concat)
# 残差层
otherpollution_residual_connection = Dense(
    8,
    activation=keras.layers.LeakyReLU(alpha=0.4),
    name="ResidualConnectionOPA")(otherpollution_x)
otherpollution_residual_connection = Dense(
    24,
    activation=keras.layers.LeakyReLU(alpha=0.4),
    name="FullConnectionOPA_RC")(otherpollution_residual_connection)
otherpollution_residual_connection = add(
    [otherpollution_x, otherpollution_residual_connection], name="ResidualConnectionOPA_Add")
# 全连接层
otherpollution_x = Dense(
    6,
    activation=keras.layers.LeakyReLU(alpha=0.4),
    name="FullConnectionOPA_2")(otherpollution_residual_connection)
# Dropout
otherpollution_y = core.Dropout(
    rate=0.5, name="OtherPollution_Module")(otherpollution_x)
# 模型层
model_otherpollution = Model(
    inputs=[
        OtherPollution_input,
        AODs_input],
    outputs=otherpollution_y)

###### 第四个模块 时空元属性模块
# 输入层
TimeStation_input = Input(shape=(len(data_ts_train.columns),), name="TimeStation_input")

# 因素融合层
timestation_concat = concatenate([TimeStation_input, AODs_input])
# 全连接层
timestation_x = Dense(
    24,
    activation=keras.layers.LeakyReLU(alpha=0.4),
    name="FullConnectionOTSA_1")(timestation_concat)
# 残差层
timestation_residual_connection = Dense(
    8,
    activation=keras.layers.LeakyReLU(alpha=0.4),
    name="ResidualConnectionTSA")(timestation_x)
timestation_residual_connection = Dense(
    24,
    activation=keras.layers.LeakyReLU(alpha=0.4),
    name="FullConnectionTSA_RC")(timestation_residual_connection)
timestation_residual_connection = add(
    [timestation_x, timestation_residual_connection], name="ResidualConnectionTSA_Add")
# 全连接层
timestation_x = Dense(
    8,
    activation=keras.layers.LeakyReLU(alpha=0.4),
    name="FullConnectionTSA_2")(timestation_residual_connection)
timestation_x = Dense(
    4,
    activation=keras.layers.LeakyReLU(alpha=0.4),
    name="FullConnectionTSA_222")(timestation_x)
# Dropout
timestation_y = core.Dropout(
    rate=0.5, name="TimeStation_Module")(timestation_x)
# 模型层
model_timestation = Model(
    inputs=[
        TimeStation_input,
        AODs_input],
    outputs=timestation_y)
# 第五个模块 整体影响模块
# 因素融合层
allin_concat = concatenate([Meteorology_input,
                            Weather_input,
                            OtherPollution_input,
                            TimeStation_input,
                            AODs_input])
# 全连接层
allin_x = Dense(
    48,
    activation=keras.layers.LeakyReLU(alpha=0.2),
    name="FullConnectionAIA_1")(allin_concat)
# 残差层
allin_residual_connection = Dense(
    24,
    activation=keras.layers.LeakyReLU(alpha=0.2),
    name="ResidualConnectionAIA")(allin_x)
allin_residual_connection = Dense(
    48,
    activation=keras.layers.LeakyReLU(alpha=0.2),
    name="FullConnectionAIA_RC")(allin_residual_connection)
allin_residual_connection = add(
    [allin_x, allin_residual_connection], name="ResidualConnectionAIA_Add")
# 全连接层
allin_x = Dense(18, activation=keras.layers.LeakyReLU(alpha=0.2),
                name="FullConnectionAIA_2")(allin_residual_connection)
# Dropout
allin_y = core.Dropout(rate=0.6, name="AllIn_Module")(allin_x)
# 模型层
model_allin = Model(
    inputs=[
        Meteorology_input,
        Weather_input,
        OtherPollution_input,
        TimeStation_input,
        AODs_input],
    outputs=allin_y)

# 最后的融合
# 捕捉的影响的融合层
res_concat = concatenate([model_meteorology.output,
                          model_weather.output,
                          model_otherpollution.output,
                          model_timestation.output,
                          model_allin.output])
# 全连接层
res_x = Dense(
    64,
    name="ResFullConnectionResModelForLast")(res_concat)
# 残差连接层
res_residual_connection = Dense(
    48,
    name="ResidualConnectionLast")(res_x)
res_residual_connection = Dense(
    64,
    name="FullConnectionLast_RC")(res_residual_connection)
res_residual_connection = add(
    [res_x, res_residual_connection], name="ResidualConnectionLast_Add")
# 全连接层
res_x = Dense(24, activation=keras.layers.ELU(alpha=0.4),
              name="FullConnectionLast_2")(res_residual_connection)
res_x2 = Dense(8, activation=keras.layers.ELU(alpha=0.4),
              name="FullConnectionLast_2x")(res_x)
# Dropout
res_y = core.Dropout(rate=0.1, name="Res_Module")(res_x2)
# sigmoid 最终融合结果
res_outcome = Dense(
    1,
    activation='sigmoid',
    kernel_regularizer=keras.regularizers.l2(0.01),
    name="sigmoid_FC")(res_y)
# 编译最终模型
model_last = Model(
    inputs=[
        Meteorology_input,
        Weather_input,
        OtherPollution_input,
        TimeStation_input,
        AODs_input],
    outputs=res_outcome)
model_last.compile(loss="mse",
                   optimizer=keras.optimizers.Adam(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
                   metrics=["accuracy"])


# print(model_last.summary())
model_last.fit([data_sky_train,data_t1_train,data_ndvi_train,data_ts_train,data_aod_train],data_pm_train,
               epochs = 2000,
               batch_size =512)

res = model_last.predict([data_sky_test,data_t1_test,data_ndvi_test,data_ts_test,data_aod_test])
datares = res - data_pm_test
datares.PM25 = datares.PM25.map(lambda x: abs(x))
print(np.average(datares))

data_predt = pd.concat([datares,data_pm_test], axis=1)
data_predt.columns = ["差值",'真']
data_predt['差值'] = data_predt['差值'].map(lambda x: abs(x))
data_predt['百分误'] = data_predt['差值'].div(data_predt["真"])


D:\DataScience\Anaconda\lib\site-packages\keras\activations.py:211: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))
W0903 20:46:11.766966 19412 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
D:\DataScience\Anaconda\lib\site-packages\keras\activations.py:211: UserWarning: Do not pass a layer instance (such as ELU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


Epoch 1/2000
19892/19892 [==============================] - 6s 313us/step - loss: 0.3300 - acc: 0.0000e+00
Epoch 2/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.4010 - acc: 0.0000e+00
Epoch 3/2000
19892/19892 [==============================] - 0s 17us/step - loss: 0.4342 - acc: 1.0054e-04
Epoch 4/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.7313 - acc: 0.0000e+00
Epoch 5/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.7444 - acc: 0.0000e+00
Epoch 6/2000
19892/19892 [==============================] - 0s 17us/step - loss: 0.7272 - acc: 0.0000e+00
Epoch 7/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.7487 - acc: 0.0000e+00
Epoch 8/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.7447 - acc: 5.0271e-05
Epoch 9/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.7377 - acc: 0.0000e+00
Epoch 10/2000
19892/19892 [==================

19892/19892 [==============================] - 0s 16us/step - loss: 0.1901 - acc: 1.5081e-04
Epoch 78/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.1903 - acc: 1.5081e-04
Epoch 79/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.1879 - acc: 1.5081e-04
Epoch 80/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.1884 - acc: 1.5081e-04
Epoch 81/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.1879 - acc: 1.5081e-04
Epoch 82/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.1878 - acc: 1.5081e-04
Epoch 83/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.1879 - acc: 1.5081e-04
Epoch 84/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.1868 - acc: 1.5081e-04
Epoch 85/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.1865 - acc: 1.5081e-04
Epoch 86/2000
19892/19892 [========================

19892/19892 [==============================] - 0s 16us/step - loss: 0.1494 - acc: 1.5081e-04
Epoch 154/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.1490 - acc: 1.5081e-04
Epoch 155/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.1482 - acc: 1.5081e-04
Epoch 156/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.1478 - acc: 1.5081e-04
Epoch 157/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.1467 - acc: 1.5081e-04
Epoch 158/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.1464 - acc: 1.5081e-04
Epoch 159/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.1462 - acc: 1.5081e-04
Epoch 160/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.1451 - acc: 1.5081e-04
Epoch 161/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.1449 - acc: 1.5081e-04
Epoch 162/2000
19892/19892 [===============

19892/19892 [==============================] - 0s 16us/step - loss: 0.1216 - acc: 1.5081e-04
Epoch 229/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.1214 - acc: 1.5081e-04
Epoch 230/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.1213 - acc: 1.5081e-04
Epoch 231/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.1211 - acc: 1.5081e-04
Epoch 232/2000
19892/19892 [==============================] - 0s 15us/step - loss: 0.1207 - acc: 1.5081e-04
Epoch 233/2000
19892/19892 [==============================] - 0s 15us/step - loss: 0.1211 - acc: 1.5081e-04
Epoch 234/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.1215 - acc: 1.5081e-04
Epoch 235/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.1209 - acc: 1.5081e-04
Epoch 236/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.1208 - acc: 1.5081e-04
Epoch 237/2000
19892/19892 [===============

...

19892/19892 [==============================] - 0s 16us/step - loss: 0.4284 - acc: 1.5081e-04
Epoch 1507/2000
19892/19892 [==============================] - 0s 15us/step - loss: 0.4886 - acc: 1.0054e-04
Epoch 1508/2000
19892/19892 [==============================] - 0s 15us/step - loss: 0.4947 - acc: 1.0054e-04
Epoch 1509/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.4460 - acc: 5.0271e-05
Epoch 1510/2000
19892/19892 [==============================] - 0s 15us/step - loss: 0.4795 - acc: 0.0000e+00
Epoch 1511/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.4407 - acc: 1.0054e-04
Epoch 1512/2000
19892/19892 [==============================] - 0s 15us/step - loss: 0.5218 - acc: 5.0271e-05
Epoch 1513/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.5172 - acc: 1.0054e-04
Epoch 1514/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.5116 - acc: 5.0271e-05
Epoch 1515/2000
19892/19892 [======

Epoch 1581/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.4937 - acc: 1.5081e-04
Epoch 1582/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.4572 - acc: 5.0271e-05
Epoch 1583/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.5207 - acc: 5.0271e-05
Epoch 1584/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.4849 - acc: 1.0054e-04
Epoch 1585/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.5119 - acc: 1.0054e-04
Epoch 1586/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.4775 - acc: 1.0054e-04
Epoch 1587/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.4760 - acc: 1.0054e-04
Epoch 1588/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.4870 - acc: 1.0054e-04
Epoch 1589/2000
19892/19892 [==============================] - 0s 17us/step - loss: 0.5185 - acc: 1.0054e-04
Epoch 1590/2000
198

19892/19892 [==============================] - 0s 16us/step - loss: 0.5022 - acc: 5.0271e-05
Epoch 1657/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.5000 - acc: 1.5081e-04
Epoch 1658/2000
19892/19892 [==============================] - 0s 15us/step - loss: 0.4797 - acc: 1.0054e-04
Epoch 1659/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.4904 - acc: 1.0054e-04
Epoch 1660/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.5420 - acc: 1.0054e-04
Epoch 1661/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.5103 - acc: 5.0271e-05
Epoch 1662/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.4704 - acc: 1.0054e-04
Epoch 1663/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.5005 - acc: 5.0271e-05
Epoch 1664/2000
19892/19892 [==============================] - 0s 17us/step - loss: 0.5590 - acc: 5.0271e-05
Epoch 1665/2000
19892/19892 [======

Epoch 1731/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.4888 - acc: 5.0271e-05
Epoch 1732/2000
19892/19892 [==============================] - 0s 20us/step - loss: 0.4821 - acc: 1.0054e-04
Epoch 1733/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.4595 - acc: 1.0054e-04
Epoch 1734/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.4520 - acc: 1.5081e-04
Epoch 1735/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.5543 - acc: 5.0271e-05
Epoch 1736/2000
19892/19892 [==============================] - 0s 16us/step - loss: 0.5018 - acc: 0.0000e+00
Epoch 1737/2000
19892/19892 [==============================] - 0s 17us/step - loss: 0.4679 - acc: 1.0054e-04
Epoch 1738/2000
19892/19892 [==============================] - 0s 17us/step - loss: 0.4995 - acc: 1.0054e-04
Epoch 1739/2000
19892/19892 [==============================] - 0s 17us/step - loss: 0.4576 - acc: 1.0054e-04
Epoch 1740/2000
198

KeyboardInterrupt: 